<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюера</b></font>

Привет Данила! Меня зовут Марат, и я буду твоим ревьюером. Спешу сообщить что все ключевые этапы в работе выполнены,  с задачей тебе удалось справиться. По поводу обращения - в IT сфере принято общаться на «ты» :) Но, если привычней на «вы», дай знать. Как ревьюера моя задача помочь тебе в развитии, дав хорошие советы. Я внимательно посмотрю твой код, ознакомлюсь с твоими выводами и оставлю комментарии. Где то могу предложить небольшие исправление в коде, но ненавязчиво. Где потребуются уточнения, я оставлю много наводящих вопросов. Они помогут тебя с поиском верного решения.

Все мои комментарии размечены по цветам, для лучшего восприятия. 
    
<div class="alert alert-success">Зеленым цветом и 👍, словом «Успех» отмечены особо удачные и элегантные решения, которыми ты можешь гордиться. </div>
        
<div class="alert alert-warning">Желтым и 🤔, словом «Совет», помечены решения у которых есть альтернативные решения, более оптимальные. Ты можешь найти их сразу и доработать проект, или отложить это на потом, для будущих проектах. Проект будет принят и без их доработки. </div>
        
<div class="alert alert-danger"> Красным цветом и значком ❌, словом «Ошибка» помечу твои решения, на которые стоит обратить внимание прежде всего. После их доработки проект будет принят. </div>
        
Залог успеха - работа сообща, взаимное уважение и работа в диалоге. Поэтому, помечай свои ответные комментарии на мои реплики заметным цветом или курсивом, так мне будет легче их отслеживать. Пожалуйста, не изменяй и не удаляй мои комментарии. Все это поможет выполнить повторную проверку быстрей.

</div>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех 👍:


Вступление в работу очень важно, так человек, который смотрит твой проект (и на работе в том числе) будет сразу введен в курс дела.





</div>

## Загрузка данных

In [3]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [4]:
data = pd.read_csv('/datasets/insurance.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Проверим данные на дубликаты

In [7]:
data.duplicated().sum()

153

In [8]:
data = data.drop_duplicates()

In [9]:
data.duplicated().sum()

0

Дубликаты были удалены из датасета

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех 👍: 
    
EDA проведено.  



    
<div class="alert alert-warning">


Совет 🤔: 
    
  
    
- Скорее всего дубликатов нет, просто совпадение параметров разных людей    

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Предсказание модели не поменяется, но поменяются веса для новых признаков.

**Обоснование:**
Формула расчёта весов:

$$
w = (X^T X)^{-1} X^T y
$$

Представим новую матрицу, как умножение исходной матрицы на обратную:

$$
X1 = X * P
$$

Подставим новое значение *X* в формулу *w1*

$$
w1 = (XP^T X)^{-1} XP^T y
$$

Раскроем первые скобки

$$
w1 = (P^TX^TXP)^{-1}P^TX^t y
$$

Перегруппируем множители в скобках и раскроем их

$$
w1 = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^T y
$$

Так как по условию *P* обратимая, то произведение равно *E* (единичная матрица). Можно заметить, что справа получилась формула для *w*

$$
w1 = P^{-1}(X^X)^{-1}EX^T = P^{-1} w
$$

Подставим новую формулу *w* в функцию линейной регрессии

$$
a = Xw = X1w1 = XPP^{-1}w
$$

Так как по условию *P* обратимая, то произведение *PP<sup>-1</sup>* снова равно *E*

$$
a1 = XPP^{-1}w = XEw = Xw = a
$$

Я доказал, что предсказания *a1* для матрицы признаков, умноженнных на обратимую матрицу *P* равны предсказаниям *a*.

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>



Успех 👍:




Самая частая ошибка - не учитывают что обратная матрица может быть только у квадратной.  Часто не соблюдают правила матричного преобразования, или меняют матрицы местами при перемножении как цифры.  


У тебя все ОК. 
    


## Алгоритм преобразования

**Алгоритм**

Мы создаем квадратную матрицу, размер которой соответствует числу признаков, и наполняем ее случайными числами. Далее мы проверяем, является ли эта матрица обратимой. Если матрица обратима, мы умножаем наши признаки на эту матрицу. 

Важно отметить, что требуется, чтобы матрица была обратимой, чтобы мы могли успешно восстановить исходные данные.


<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех 👍:


Принято

<div class="alert alert-warning">

Совет 🤔:

    
    

    
Как еще можно получать обратимую матриуц?! Можно взять единичную матрицу и применить  элементарный преобразования. А еще можно использовать [матрицу по принципу диагонального доминирования](https://question-it.com/questions/13911024/generatsija-obratimyh-matrits-v-numpytensorflow) 



**Обоснование**

Мы можем ожидать, что качество линейной регрессии будет одинаковым до и после преобразования данных. Преобразование данных не влияет на способность модели предсказывать целевой признак.

## Проверка алгоритма

In [10]:
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

In [16]:
def get_rand_matrix():
    det = 0
    while det == 0:
        matrix = np.random.normal(size=(data.shape[1] - 1, data.shape[1] - 1))
        det = np.linalg.det(matrix)
    return matrix

In [17]:
matrix = get_rand_matrix()

In [18]:
matrix_inv = np.linalg.inv(matrix)
matrix_inv

array([[-8.71854722,  6.21711762,  5.17863818, -1.68229742],
       [ 0.1275935 ,  0.23109038, -0.1634978 , -0.11027591],
       [ 6.46389994, -4.80134368, -3.38746968,  0.68041472],
       [ 0.02918406, -0.57027114, -0.61224608,  0.02339108]])




<div class="alert alert-warning">
<font size="5"><b>Комментарий ревьюера</b></font>

Совет 🤔:




- если матрица необратимая выскочит ошибка на np.linalg.inv, код не исполнится до конца
   
Можно здесь было сделать похитрей - генерируем матрицу пока не получим обратимую и наш код  гарантированно полностью отработает. Добиться этого можно например так:


    
    def get_rand_matrix():
        det = 0
        while det == 0:
            matrix = np.random.normal(size=(data.shape[1], data.shape[1]))
            det = np.linalg.det(matrix)
        return matrix

C помощью рекурсии

    def func(dim):
        transform_matrix = np.random.rand(dim, dim)
        try: 
            inv = np.linalg.inv(transform_matrix)
            return transform_matrix, inv
        except:
            func(dim)  
    

- что насчет универсальности кода? Сейчас ты цифрой определяешь размерность случайной матрички, почему бы не написать код, который отработает для любой размерности матрицы обьект-признак?      
    


<div class="alert alert-info">
    <b>Комментарий студента</b><br>
    Привет! Спасибо за подсказку, использовал твой код для большей универсальности.
</div>

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Успех 👍:



👍




Привет Данила!


In [19]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
r2_score_value = r2_score(target, predictions)
print(f'{r2_score_value:.13f}')

0.4302010044852


In [20]:
features_matrix = features.dot(matrix)

In [21]:
model = LinearRegression()
model.fit(features_matrix, target)
predictions = model.predict(features_matrix)
r2_score_value = r2_score(target, predictions)
print(f'{r2_score_value:.13f}')

0.4302010044852




<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех 👍:
    
Все сошлось

<div class="alert alert-warning">


Совет 🤔: 
    


- каноны ML - когда мы моделируем, разбиваем датасет на несколько, используя train для обучения и test для проверки, опять же это никак не изменит выводы, но каноны стоит уважать
   

## Вывод

Значения обеих моделей сошлись, что говорит о том, что модель не потеряла своё качество. Данные пользователей теперь защищены!


<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюера</b></font>


Данила, у тебя старательно выполненная работа, все четко, осмысленно. Отмечаю уровень твоей программистской подготовки. 




Я оставил небольшие советы и вопросики (если есть время и желание можешь воспользоваться/ответить)
    



Посылаю на вторую итерацию, чтобы ответить на вопросы если они появятся. На связи
    









<div class="alert alert-warning">
<b>Комментарий ревьюера</b>



На мой взгляд проект очень необычный, выделяющийся из остальных проектов потому что касается раздела математики лежащей в основе машинного обучения  (помимо производных, матанализа и теории вероятности с матстатистикой). Если в дальнейшем ты углубишься в изучении нейростетей, то понимание линейной алгебры будет очень полезным, потому что, по сути, все нейросети - это перемножение матриц и Линейная алгебра даёт очень компактный способ записи того что происходит в нейросетях. 

Кроме того языком линейной алгебра можно объяснить тематическое моделирование и pca (через сингулярное разложение)

Мне не очень нравится эти ссылки, наверняка можно найти получше, но там вполне наглядно показывают связь между линейной алгебры и многослойным перцептроном
    
https://www.youtube.com/watch?v=kmdqBlls_HQ    
https://www.youtube.com/watch?v=AOZZeTQl4Xc


Это даже важно как "понятийный язык", вот здесь о модели word2vec (это NLP) рассказываться "языком линейной алгебры" с [22 минуты](https://www.youtube.com/watch?v=MBQdMQUZMQM&t=3340s) 
    

Возможно будет полезно:
    
    

    
- Если хочешь посмотреть вообще про линейную алгебру и матрицы, то мне нравится  вот это [видео](https://www.youtube.com/watch?v=RNTRYicPvWQ&list=PLVjLpKXnAGLXPaS7FRBjd5yZeXwJxZil2), рассказывается по существу и увлекательно.  


- по применению криптографии в машинном обучении на английском: [тут](https://www.cs.cmu.edu/~rjhall/JOS_revised_May_31a.pdf)




<div class="alert alert-info">
    <b>Комментарий студента</b><br>
    Спасибо за подробное ревью. Ссылки себе сохраню, в целом очень интересна тема нейронок изнутри :) 
</div>


<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюераV2</b></font>

    
    в целом очень интересна тема нейронок изнутри    
  
    
Ещё бы, и будет гораздо проще понять, если использовать язык линейной алгебры    
    
    
Ну что же спасибо за работу  Данила,  желаю успехов в дальнейшей учебе!
    

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования